In [154]:
from __future__ import division
import sqlite3
import logging
from pprint import pprint

import time

# import itertools
from sklearn.cluster import DBSCAN
# from sklearn import metrics
# from sklearn.datasets.samples_generator import make_blobs
# from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as hierarchy_clustering
from scipy.spatial.distance import pdist, squareform
# from jellyfish import jaro_distance
import numpy as np
from sacred import Experiment

from distances import lcs_distance, fuzzy_distance

db_path = '../Database/src/main/resources/test3.db'
n_of_authors =  10 ** 3

In [155]:
def get_primitive_authors(cursor, limit=1000):
    """

    :type cursor: sqlite3.Cursor
    :type limit: int
    :return: list
    """
    cursor.execute('SELECT * FROM primitive_author LIMIT %d' % limit)
    primitive_authors = cursor.fetchall()
    return primitive_authors

In [156]:
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
primitive_authors = get_primitive_authors(cursor, n_of_authors)

In [157]:
def get_slm_list(authors, distance=fuzzy_distance, eps=0.1):
    def distance_func(i, j, key='surname'):
        i = int(i[0])
        j = int(j[0])
        return distance(authors[i][key], authors[j][key])

    X = np.array([[i] for i in xrange(len(authors))])
    
    tic2 = time.time()
    Y = pdist(X, distance_func)
    print "pdist: %f" % (time.time() - tic2)
    
    distance_matrix = squareform(Y)
    distance_matrix[np.tril_indices_from(distance_matrix)] = 1
    indices = zip(*np.where(distance_matrix < eps))
    return indices

tic1 = time.time()
slm_pairs =  get_slm_list(primitive_authors)
print "Outer timer: %f" % (time.time() - tic1)
print "N of connections: %d" % len(slm_pairs)
# for i, j in slm_pairs:
#     print "%d\t%d" % (i, j)

pdist: 33.035363
Outer timer: 33.090378
N of connections: 45


In [158]:
with open('slm_input.txt', 'w') as output:
    for i, j in slm_pairs:
        output.write("%d\t%d\n" % (i, j))

In [159]:
clusters = {}
with open('slm_output.txt') as input_file:
    for i, cluster_id in enumerate(input_file):
        cluster_id = cluster_id.strip()
        author_id = primitive_authors[i]['_id']
        author_surname = primitive_authors[i]['surname']
        try:
            clusters[cluster_id] += [(author_id, author_surname)]
        except KeyError:
            clusters[cluster_id] = [(author_id, author_surname)]

pprint(filter(lambda (k, v): len(v) >= 2, clusters.items()))

[('0',
  [(143, u'Xu, Chen'),
   (226, u'Chen, Xi'),
   (306, u'Chen, Xiu'),
   (705, u'Cao, Xu-Chen')]),
 ('32', [(939, u'Hashemi, M'), (943, u'Hashemi, S M')]),
 ('31', [(527, u'Zhang, Hong-Sheng'), (832, u'Zhang, Congsheng')]),
 ('30', [(32, u'Xu, Tao'), (761, u'Xu, Tao')]),
 ('6',
  [(532, u'Huang, Ying-Hui'), (735, u'Huang, Qing'), (969, u'Huang, Ying')]),
 ('1', [(162, u'Zhang, Min'), (426, u'Zhang, Min'), (500, u'Zhang, Min')]),
 ('24', [(301, u'Xu, Yong'), (646, u'Xu, Yong')]),
 ('25', [(658, u'Zhang, Hao'), (977, u'Zhang, Hao')]),
 ('26', [(529, u'Li, Hu'), (696, u'Li, Hui')]),
 ('27', [(518, u'Yang, Ming-Xia'), (715, u'Xia, Yang')]),
 ('20', [(59, u'Zhang, Heng-Hui'), (594, u'Zhang, Hui')]),
 ('21', [(625, u'Li, Qiang'), (760, u'Li, Qiang')]),
 ('22', [(626, u'Wang, Hui'), (862, u'Wang, Hui')]),
 ('23', [(134, u'Zhu, Zhaowei'), (638, u'Zou, Zhaowei')]),
 ('28', [(732, u'Liu, Hai-Ying'), (770, u'Liu, Ying')]),
 ('29', [(595, u'Wang, Jing'), (756, u'Wang, Rujing')]),
 ('7', [(7

In [2]:
a = 132

In [3]:
print a

132
